In [2]:
%matplotlib inline
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [3]:
df = pd.read_csv('python/data/train.csv')

In [26]:
df.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,annual_inc,loan_status,pymnt_plan,...,MORTGAGE,NONE,OTHER,OWN,RENT,Not Verified,Source Verified,Verified,issue_d_le,earliest_cr_line_le
0,15000.0,0,11.99,498.15,1,7,4,70000.0,1,0,...,1,0,0,0,0,0,0,1,2013.750000,1991.916667
1,3725.0,0,6.03,113.38,0,0,0,52260.0,1,0,...,1,0,0,0,0,0,1,0,2012.750000,2000.750000
2,16000.0,0,11.14,524.89,1,6,3,67500.0,1,0,...,0,0,0,0,1,0,1,0,2013.250000,2001.500000
3,4200.0,0,13.33,142.19,2,12,1,21600.0,0,0,...,1,0,0,0,0,1,0,0,2015.166667,2003.333333
4,6500.0,0,12.69,218.05,1,9,10,41000.0,1,0,...,0,0,0,0,1,1,0,0,2012.000000,1990.666667


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200189 entries, 0 to 200188
Data columns (total 43 columns):
loan_amnt                     200189 non-null float64
term                          200189 non-null int64
int_rate                      200189 non-null float64
installment                   200189 non-null float64
grade                         200189 non-null int32
sub_grade                     200189 non-null int32
emp_length                    200189 non-null int32
annual_inc                    200189 non-null float64
loan_status                   200189 non-null int64
pymnt_plan                    200189 non-null int32
purpose                       200189 non-null int32
zip_code                      200189 non-null int32
addr_state                    200189 non-null int32
dti                           200189 non-null float64
delinq_2yrs                   200189 non-null float64
inq_last_6mths                200189 non-null float64
mths_since_last_delinq        200189 non-nu

In [6]:
df.emp_length.fillna(value=0,inplace=True)
df.revol_util.fillna(value=0,inplace=True)
df.collections_12_mths_ex_med.fillna(value=df.collections_12_mths_ex_med.mean(),inplace=True)
df.mths_since_last_delinq.fillna(value=df.mths_since_last_delinq.mean(),inplace=True)
df.tot_coll_amt.fillna(value=df.tot_coll_amt.mean(),inplace=True)
df.tot_cur_bal.fillna(value=df.tot_coll_amt.mean(),inplace=True)
df.total_rev_hi_lim.fillna(value=df.total_rev_hi_lim.mean(),inplace=True)
df['emp_length'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
df['emp_length'] = df['emp_length'].astype(int)
df['term'] = pd.factorize(df['term'])[0]

In [7]:
df['is_title_known'] = df['emp_title'].map(lambda x: 0 if x == 'n/a' else 1)
df.drop('emp_title', axis=1, inplace=True)

In [8]:
df['zip_code'] = df['zip_code'].where(df['zip_code'].str.len() == 4, 
                                               df['zip_code'].str[:3])
df['zip_code'] = df['zip_code'].astype(int)

In [9]:
label_encoder = LabelEncoder()
categorical_columns = df[['grade','sub_grade','pymnt_plan','initial_list_status','purpose','application_type','addr_state']]
for column in categorical_columns:
    df[column] = label_encoder.fit_transform(df[column])

In [10]:
one_hot_encoded_home_ownership = pd.get_dummies(df.home_ownership)
df = df.drop('home_ownership',axis = 1)

df = df.join(one_hot_encoded_home_ownership)

In [11]:
one_hot_encoded_verification_status = pd.get_dummies(df.verification_status)
df = df.drop('verification_status',axis = 1)

df = df.join(one_hot_encoded_verification_status)

In [12]:
def month_to_decimal(month):
    month_dict = {'Jan':0, 'Feb':1/12., 'Mar':2/12., 'Apr':3/12., 'May':4/12., 'Jun':5/12., 
     'Jul':6/12., 'Aug':7/12., 'Sep':8/12., 'Oct':9/12., 'Nov':10/12., 'Dec':11/12.}
    return month_dict[month]

def convert_date(month_year):
    month_and_year = month_year.split('-')
    return float(month_and_year[1]) + month_to_decimal(month_and_year[0])

def encode_with_func(df, column_name, func_name):
    df[column_name+'_le'] = df[column_name].map(func_name)
    df.drop(column_name, axis=1, inplace=True)

encode_with_func(df, 'issue_d', convert_date)

In [13]:
encode_with_func(df, 'earliest_cr_line', convert_date)

In [14]:
Y = df['loan_status'].values

X = df.drop(['loan_status'], axis=1)

from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
X, Y = ros.fit_resample(X, Y)

x_train, x_test, y_train, y_test = model_selection.train_test_split(X, Y, test_size=0.3, random_state=42)

In [15]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.fit_transform(x_test)

In [16]:
tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf = 5)
tree.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=5, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [31]:
from sklearn.model_selection import GridSearchCV, cross_val_score

tree_params = {'max_depth': [3,5,7,9],'min_samples_leaf': [3,5]}
tree_grid = GridSearchCV(tree, tree_params,cv=5, n_jobs=-1,verbose=True)
tree_grid.fit(x_train, y_train)
print(tree_grid.best_params_)
print(tree_grid.best_score_)
accuracy_score(y_test, tree_grid.predict(x_test))

Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:   21.1s finished


{'max_depth': 9, 'min_samples_leaf': 3}
0.655514511873351


0.6554171975545253

In [17]:
y_tree = tree.predict(x_test)

In [18]:
print("Accuracy:",metrics.accuracy_score(y_test, y_tree))
print("Precision:",metrics.precision_score(y_test, y_tree))
print("Recall:",metrics.recall_score(y_test, y_tree))

Accuracy: 0.640534492542587
Precision: 0.6557195136347539
Recall: 0.5938970147977076


In [19]:
y_pred_tree = tree.predict_proba(x_test)[::,1]
auc_tree = metrics.roc_auc_score(y_test, y_pred_tree)
print(auc_tree)

0.6930181389263298


In [20]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [21]:
knn_pred = knn.predict(x_test)

In [22]:
print("Accuracy:",metrics.accuracy_score(y_test, knn_pred))
print("Precision:",metrics.precision_score(y_test, knn_pred))
print("Recall:",metrics.recall_score(y_test, knn_pred))
y_pred_knn = knn.predict_proba(x_test)[::,1]
auc_knn = metrics.roc_auc_score(y_test, y_pred_knn)
print(auc_knn)

Accuracy: 0.8520937503345932
Precision: 0.931440842193474
Recall: 0.76058506543495
0.8522222820366266


In [23]:
forest = RandomForestClassifier(n_estimators=100, min_samples_leaf= 5, max_depth=5)
forest.fit(x_train, y_train)
y_forest = forest.predict(x_test)

In [24]:
print("Accuracy:",metrics.accuracy_score(y_test, y_forest))
print("Precision:",metrics.precision_score(y_test, y_forest))
print("Recall:",metrics.recall_score(y_test, y_forest))
y_pred_forest = forest.predict_proba(x_test)[::,1]
auc_forest = metrics.roc_auc_score(y_test, y_pred_forest)
print(auc_forest)

Accuracy: 0.6448601132798698
Precision: 0.6588460729558573
Recall: 0.602899666410059
0.7050455518697153
